In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.25 µs


In [2]:
# typically takes 30 secs
from nlp_metrics import *

time: 21.9 s


In [3]:
database = load_pickle('../big_data/database2.pickle')
database = list(set([word for word in database if word!= '-PRON-']))
print(len(database))

2005
time: 28.3 ms


In [4]:
class ev(evaluation):
    def __init__(self, filename, tag):
        self.dic = self.load_dic({}, filename, tag)
        self.ori = tag
        self.gens = []

    def ingr_f1(self, root = True):
        value, number = [], []
        for i, v in tqdm.tqdm(self.dic.items()):
            true, pred = v['%s_ingr'%(self.ori)], v['%s_ingr'%(self.gen)]
            if root:
                true, pred = self.ingr(true), self.ingr(pred)
            scores = metrics(true, pred)
            value.append(scores.f1())
            number.append(len(set(pred)))
        avg = sum(value)/len(value)
        print(avg)
        print('average ingr number', sum(number)/len(number))
        return avg
    
    def ingr_precision_recall(self, generate = 'ingr'):
        assert generate in ['ingr','instr','human']
        precision, recall = [], []
        for i, v in tqdm.tqdm(self.dic.items()):
            if generate == 'ingr':
                true, pred = v['%s_ingr'%(self.ori)], v['%s_ingr'%(self.gen)]
                true, pred = self.ingr(true), self.ingr(pred)
            elif generate == 'instr':
                true, pred = v['%s_ingr'%(self.ori)], v['%s_instr'%(self.gen)]
                true, pred = self.ingr(true), self.instr(pred)
            elif generate == 'human':   
                true, pred = v['%s_ingr'%(self.ori)], v['%s_instr'%(self.ori)]
                true, pred = self.ingr(true), self.instr(pred)
            scores = metrics(true, pred)
            precision.append(scores.precision())
            recall.append(scores.recall())
    
        print('precision', sum(precision)/len(precision))
        print('recall', sum(recall)/len(recall))
    
    def instr(self, directions):
        instr = sp.spacy(directions)
        root_instr = []
        for chunk in instr.noun_chunks:
            idx_rootnoun = chunk.end - 1
            str_rootnoun = instr[idx_rootnoun].lemma_
            if str_rootnoun in database:
                root_instr.append(str_rootnoun)
        return root_instr
    
    def ingr(self, lst):
        '''
        Args: lst: a list of ingredient names
        used when len(lst) must equal to root_match
        '''
        hl = [[{'text':x, 'highlight': None} for x in i.split(' ')] for i in lst]
        root_match = []
        for i, ingr in enumerate(lst):
            if ' ' not in ingr:
                hl[i][0]['highlight'] = 'wrong'
                doc = sp.spacy(ingr)
                root_match.append(doc[0].lemma_)
            else:
                phrase = 'Mix the %s and water.'%ingr
                doc = sp.spacy(phrase)
                
                last_chunk = None
                for chunk in doc.noun_chunks:
                    if chunk.text != 'water':
                        last_chunk = chunk
                if not last_chunk:
                    root_match.append('CANNOT_DETECT')
                else:
                    found = False
                    for j, word in enumerate(hl[i]):
                        if doc[last_chunk.end - 1].text in word['text']:
                            hl[i][j]['highlight'] = 'wrong' 
                            root_match.append(doc[last_chunk.end - 1].lemma_)
                            found = True
                            break
                    if not found:
                        root_match.append('CANNOT_DETECT')
                        
        assert len(root_match) == len(lst)
        return root_match

    '''
    exporting data
    '''
    def to_bleu(self):
        to_write = {'%s_i'%(self.ori):'',
                    '%s_i'%(self.gen):'',
                    '%s_d'%(self.ori):'',
                    '%s_d'%(self.gen):''}
        
        for i, v in self.dic.items():
            to_write['%s_i'%(self.ori)] += self.add_space(' $ '.join(v['%s_ingr'%(self.ori)]))+ ' $ \n'
            to_write['%s_i'%(self.gen)] += self.add_space(' $ '.join(v['%s_ingr'%(self.gen)])) + ' $ \n'
            
            to_write['%s_d'%(self.ori)] += self.add_space(v['%s_instr'%(self.ori)])+ '\n'
            to_write['%s_d'%(self.gen)] += self.add_space(v['%s_instr'%(self.gen)])+ '\n'
        
        for k, v in to_write.items():
            save('../../to_gpt2/generation_%s.txt'%(k), v ,overwrite = True)
        !eval {"perl multi-bleu.perl ../../to_gpt2/generation_%s_i.txt < ../../to_gpt2/generation_%s_i.txt" %(self.ori, self.gen)}
        !eval {"perl multi-bleu.perl ../../to_gpt2/generation_%s_d.txt < ../../to_gpt2/generation_%s_d.txt" %(self.ori, self.gen)}
    
        !eval {"rouge -f ../../to_gpt2/generation_%s_i.txt ../../to_gpt2/generation_%s_i.txt --avg"%(self.ori, self.gen)}
        !eval {"rouge -f ../../to_gpt2/generation_%s_d.txt ../../to_gpt2/generation_%s_d.txt --avg"%(self.ori, self.gen)}
        
        print()

time: 40.9 ms


In [5]:
data = ev('../../to_gpt2/recipe1M_1218/test/y/', 'ori')
data.append_dic('../../to_gpt2/generation_1220_k3_test/', '117M')
data.append_dic('../../to_gpt2/generation_scratch_k3_test/', 'scratch')
data.append_dic('../../to_gpt2/generation_medium_k3_test/', '345M')

load ../../to_gpt2/recipe1M_1218/test/y/
load ../../to_gpt2/generation_1220_k3_test/
load ../../to_gpt2/generation_scratch_k3_test/
load ../../to_gpt2/generation_medium_k3_test/
time: 3.23 s


In [17]:
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.to_bleu()

117M
saved ../../to_gpt2/generation_ori_i.txt
saved ../../to_gpt2/generation_117M_i.txt
saved ../../to_gpt2/generation_ori_d.txt
saved ../../to_gpt2/generation_117M_d.txt
BLEU = 32.82, 83.6/63.9/33.7/13.0 (BP=0.839, ratio=0.850, hyp_len=84162, ref_len=98982)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 8.34, 53.6/18.5/6.8/2.8 (BP=0.708, ratio=0.743, hyp_len=381274, ref_len=513062)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tok

In [6]:
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.ingr_f1(root=True)

  0%|          | 0/4000 [00:00<?, ?it/s]

117M


  0%|          | 1/4000 [00:00<09:13,  7.22it/s]

0.7578521365259253
average ingr number 7.81975
scratch


  0%|          | 1/4000 [00:00<09:28,  7.03it/s]

0.7463840283654203
average ingr number 7.594
345M


100%|██████████| 4000/4000 [10:42<00:00,  8.19it/s]

0.7668857387552463
average ingr number 7.88125
time: 32min 1s


In [18]:
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.instr_tree()

  0%|          | 0/4000 [00:00<?, ?it/s]

117M


  0%|          | 0/4000 [00:00<?, ?it/s]

0.5197623961367492
scratch


  0%|          | 0/4000 [00:00<?, ?it/s]

0.5231892865324351
345M


100%|██████████| 4000/4000 [24:16<00:00,  3.14it/s]

0.5229560768790594
time: 1h 12min 30s


In [15]:
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.to_bleu()

k1
saved ../../to_gpt2/generation_ori_i.txt
saved ../../to_gpt2/generation_k1_i.txt
saved ../../to_gpt2/generation_ori_d.txt
saved ../../to_gpt2/generation_k1_d.txt
BLEU = 33.51, 87.7/69.7/37.7/14.9 (BP=0.779, ratio=0.800, hyp_len=79877, ref_len=99866)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 9.81, 57.4/22.7/10.0/4.9 (BP=0.618, ratio=0.675, hyp_len=350990, ref_len=519837)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokeniza

In [14]:
for tag in data.gens:
    print(tag)
    data.gen = tag
    data.instr_tree()

  0%|          | 0/4000 [00:00<?, ?it/s]

k1


  0%|          | 0/4000 [00:00<?, ?it/s]

0.5115819243870295
k3


  0%|          | 0/4000 [00:00<?, ?it/s]

...

0.5278224521629334
k10


  0%|          | 1/4000 [00:00<10:03,  6.63it/s]

0.5333886852206743
k30


100%|██████████| 4000/4000 [28:09<00:00,  3.25it/s]

0.5401450212720956
time: 2h 7min 42s
